In [1]:
%autoreload 2

In [2]:
from collections import defaultdict
import copy
import itertools
import os
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm.notebook as tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS


In [3]:
fitness_df = utils.load_fitness_data()
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples.pddl'
 'ast-mle-samples-large-best.pddl' 'ast-best-mle-regrowth-samples.pddl']


,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,mean_depth_setup,mean_depth_constraints,mean_depth_terminal,mean_depth_scoring,node_count_setup,node_count_constraints,node_count_terminal,node_count_scoring,real,original_game_name
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1.0,1.0,1.0,1.00,1.0,0.5,...,5.739130,8.538462,2.6,1.500000,23,78,10,4,True,6172feb1665491d1efbce164-0
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1.0,1.0,1.0,1.00,1.0,1.0,...,3.000000,8.710843,2.6,4.921053,10,83,10,38,True,5f77754ba932fb2c4ba181d8-2
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1.0,1.0,1.0,0.00,1.0,0.0,...,0.000000,5.611111,0.0,2.500000,0,18,0,6,True,614b603d4da88384282967a7-3
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1.0,1.0,1.0,0.00,1.0,1.0,...,0.000000,9.215686,0.0,1.500000,0,51,0,4,True,5bc79f652885710001a0e82a-5
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1.0,1.0,1.0,0.25,1.0,1.0,...,5.264706,9.622449,0.0,4.531250,34,98,0,32,True,614dec67f6eb129c3a77defd-6


In [4]:
def base_syntethic_df(df: pd.DataFrame, 
    synthetic_data_src_files: typing.Sequence[str] = ('ast-mle-samples-large-best.pddl', 'ast-best-mle-regrowth-samples.pddl'),
    positives_src_file: str = 'ast-mle-samples-large-best.pddl') -> pd.DataFrame:
    syntethic_df = fitness_df[fitness_df.src_file.isin(synthetic_data_src_files)].reset_index(drop=True)
    syntethic_df.loc[syntethic_df.src_file == positives_src_file, 'real'] = 1
    return syntethic_df

syntethic_fitness_df = base_syntethic_df(fitness_df)

In [5]:
syntethic_fitness_df.head()

,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,mean_depth_setup,mean_depth_constraints,mean_depth_terminal,mean_depth_scoring,node_count_setup,node_count_constraints,node_count_terminal,node_count_scoring,real,original_game_name
0,1666,ast-mle-samples-large-best.pddl,game-id-36,medium-objects-room-v1,1.0,0.400000,0.5,0.250000,0.5,0.5,...,8.022727,9.560440,1.571429,2.000,44,91,7,5,1,game-id-36
1,1667,ast-mle-samples-large-best.pddl,game-id-43,many-objects-room-v1,1.0,0.400000,0.5,0.000000,1.0,0.0,...,6.066667,7.128571,3.357143,2.000,30,70,14,5,1,game-id-43
2,1668,ast-mle-samples-large-best.pddl,game-id-58,few-objects-room-v1,1.0,0.363636,0.5,0.333333,1.0,0.5,...,9.000000,7.608696,4.904762,3.875,40,69,21,16,1,game-id-58
3,1669,ast-mle-samples-large-best.pddl,game-id-64,many-objects-room-v1,1.0,0.500000,1.0,0.250000,1.0,1.0,...,11.217391,7.848485,4.058824,0.000,69,33,17,1,1,game-id-64
4,1670,ast-mle-samples-large-best.pddl,game-id-65,many-objects-room-v1,0.0,0.000000,1.0,0.000000,1.0,0.0,...,2.500000,3.500000,1.571429,4.500,8,8,7,16,1,game-id-65


In [ ]:
fake_games = syntethic_fitness_df[syntethic_fitness_df.src_file == 'ast-best-mle-regrowth-samples.pddl'].reset_index()
broadcasted_original = syntethic_fitness_df.loc[[syntethic_fitness_df.index[(syntethic_fitness_df.game_name == original_name)][0] for original_name in fake_games.original_game_name], :].reset_index()

original_regrown_diffs = (broadcasted_original.drop(NON_FEATURE_COLUMNS, axis=1) - fake_games.drop(NON_FEATURE_COLUMNS, axis=1))

unchanged_games_prop = (original_regrown_diffs.drop('index', axis=1) == 0).all(axis=1).sum() / len(original_regrown_diffs)
print(f'In {unchanged_games_prop * 100:.2f}% of the games, the regrown game was identical to the original game.')

# Synthetic model-fitting experiment approach
We have a large dataset now, I can try to cross-validate over some of the choices I might make:
* Loss function
* Parameters of each loss function (margin, etc.)
* Batch size
* Number of negative examples sampled per iteration
* Regularization strength?
* ...?



In [9]:
def cross_validate(train: pd.DataFrame, feature_columns: typing.List[str], \
    param_grid: typing.Union[typing.List[typing.Dict[str, typing.Any]], typing.Dict[str, typing.Any]],
    scoring_function: typing.Callable = utils.evaluate_fitness,
    model_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    train_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None, 
    cv_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    n_folds: int = 5, verbose: int = 0):

    if model_kwargs is None:
        model_kwargs = {}

    if train_kwargs is None:
        train_kwargs = {}

    if cv_kwargs is None:
        cv_kwargs = {}

    if 'n_jobs' not in cv_kwargs: 
        cv_kwargs['n_jobs'] = -1
    if 'verbose' not in cv_kwargs:
        cv_kwargs['verbose'] = verbose

    train_tensor = utils.df_to_tensor(train, feature_columns)
    pipeline = Pipeline(steps=[('scaler', utils.CustomSklearnScaler()), ('fitness', utils.SklearnFitnessWrapper(model_kwargs=model_kwargs, train_kwargs=train_kwargs))])

    if isinstance(param_grid, list):
        for param_grid_dict in param_grid:
            param_grid_dict['fitness__n_features'] = [len(feature_columns)]
    else:
        param_grid['fitness__n_features'] = [len(feature_columns)]        

    random_seed = train_kwargs['random_seed'] if 'random_seed' in train_kwargs else None

    cv = GridSearchCV(pipeline, param_grid, scoring=scoring_function, 
        cv=KFold(n_folds, shuffle=True, random_state=random_seed), 
        **cv_kwargs)
    return cv.fit(train_tensor, None)


def synthetic_model_fitting_experiment(df: pd.DataFrame,
    param_grid: typing.Union[typing.List[typing.Dict[str, typing.Any]], typing.Dict[str, typing.Any]], 
    feature_columns: typing.Optional[typing.List[str]] = None, 
    random_seed: int = utils.DEFAULT_RANDOM_SEED,
    scoring_function: typing.Callable = utils.evaluate_fitness,
    model_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    train_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    cv_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    n_folds: int = 5, verbose: int = 0
    ):

    if model_kwargs is None:
        model_kwargs = {}

    if train_kwargs is None:
        train_kwargs = {}

    syntethic_fitness_df = base_syntethic_df(df)

    if feature_columns is None:
        feature_columns = [c for c in syntethic_fitness_df.columns if c not in NON_FEATURE_COLUMNS]

    # rng = np.random.default_rng(random_seed)

    train_df, test_df = utils.train_test_split_by_game_name(syntethic_fitness_df, random_seed=random_seed)
    cv = cross_validate(train_df, feature_columns, param_grid, 
        scoring_function=scoring_function,
        train_kwargs={'random_seed': random_seed, **train_kwargs}, 
        model_kwargs=model_kwargs, cv_kwargs=cv_kwargs, n_folds=n_folds, verbose=verbose)
    # best_model = cv.best_estimator_.named_steps['fitness'].model  # type: ignore

    # TODO: add evaluation on the held-out part of the dataset here
    return cv, test_df



In [10]:
test_param_grid = [
    {
        'fitness__loss_function': [utils.fitness_hinge_loss],
        'fitness__weight_decay': [0.0, 0.125, 0.25, 0.5, 1],  
        'fitness__margin': [1, 2, 4],
        'fitness__lr': [1e-2, 3e-3, 1e-3, 3e-4],
        'fitness__k': [4, 8, 16, 32, 64],
        'fitness__batch_size': [1, 4, 8, 16],
    },
    # TODO: add other loss functions
]

all_feature_columns = [c for c in syntethic_fitness_df.columns if c not in NON_FEATURE_COLUMNS]
model_kwargs = dict(output_activation=nn.Identity())
train_kwargs = dict()
cv_kwargs = dict(refit='overall_ecdf')
scoring = utils.build_multiple_scoring_function(
    [utils.evaluate_fitness_overall_ecdf, utils.evaluate_fitness_single_game_rank],
    ['overall_ecdf', 'single_game_rank'],
)

cv, test_df = synthetic_model_fitting_experiment(fitness_df, test_param_grid,
    scoring_function=scoring, verbose=1, 
    model_kwargs=model_kwargs, train_kwargs=train_kwargs, cv_kwargs=cv_kwargs)


Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


In [30]:
cv_df = pd.concat([
    pd.DataFrame(cv.cv_results_["params"]), 
    pd.DataFrame(cv.cv_results_["mean_test_overall_ecdf"], columns=['ecdf_mean']), 
    pd.DataFrame(cv.cv_results_["std_test_overall_ecdf"], columns=['ecdf_std']), 
    pd.DataFrame(cv.cv_results_["rank_test_overall_ecdf"], columns=['ecdf_rank']),
    pd.DataFrame(cv.cv_results_["mean_test_single_game_rank"], columns=['game_rank_mean']), 
    pd.DataFrame(cv.cv_results_["std_test_single_game_rank"], columns=['game_rank_std']), 
    pd.DataFrame(cv.cv_results_["rank_test_single_game_rank"], columns=['game_rank_rank']),
],axis=1)

cv_df.sort_values(by='ecdf_rank').head(10)

,fitness__batch_size,fitness__k,fitness__loss_function,fitness__lr,fitness__margin,fitness__n_features,fitness__weight_decay,ecdf_mean,ecdf_std,ecdf_rank,game_rank_mean,game_rank_std,game_rank_rank
300,4,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.000,-0.458353,0.007644,1,0.495898,0.020880,49
61,1,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.125,-0.458738,0.005381,2,0.500293,0.016179,5
1,1,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.125,-0.459210,0.005705,3,0.498828,0.016692,10
301,4,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.125,-0.459495,0.005939,4,0.494043,0.015016,99
380,4,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.003,2,637,0.000,-0.459534,0.005647,5,0.493945,0.017512,102
65,1,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,2,637,0.000,-0.459558,0.005915,6,0.496289,0.018368,38
362,4,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.250,-0.459609,0.005819,7,0.496582,0.018495,34
605,8,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,2,637,0.000,-0.459648,0.006021,8,0.493457,0.021437,115
60,1,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.000,-0.459669,0.005457,9,0.502734,0.015748,3
302,4,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.010,1,637,0.250,-0.459715,0.005444,10,0.494629,0.017258,78


In [31]:
cv_df.sort_values(by='game_rank_rank').head(10)

,fitness__batch_size,fitness__k,fitness__loss_function,fitness__lr,fitness__margin,fitness__n_features,fitness__weight_decay,ecdf_mean,ecdf_std,ecdf_rank,game_rank_mean,game_rank_std,game_rank_rank
120,1,16,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.000,-0.460808,0.005144,57,0.503613,0.014689,1
0,1,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.000,-0.460355,0.005679,27,0.502832,0.014760,2
60,1,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.000,-0.459669,0.005457,9,0.502734,0.015748,3
190,1,32,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,4,637,0.000,-0.461133,0.005837,80,0.500684,0.016797,4
61,1,8,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.125,-0.458738,0.005381,2,0.500293,0.016179,5
842,8,64,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.250,-0.462031,0.005791,244,0.499902,0.016398,6
125,1,16,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,2,637,0.000,-0.460739,0.005660,50,0.499219,0.016612,7
121,1,16,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.125,-0.460316,0.005429,24,0.499219,0.015212,7
180,1,32,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.000,-0.461719,0.005533,174,0.499023,0.019468,9
1,1,4,<function fitness_hinge_loss at 0x7fd4aad1cf70>,0.01,1,637,0.125,-0.459210,0.005705,3,0.498828,0.016692,10


In [14]:
syntethic_fitness_df = base_syntethic_df(fitness_df)
feature_columns = [c for c in syntethic_fitness_df.columns if c not in NON_FEATURE_COLUMNS]
train_df, test_df = utils.train_test_split_by_game_name(syntethic_fitness_df, random_seed=42)
train_df, val_df = utils.train_test_split_by_game_name(train_df, random_seed=42)
train_tensor = utils.df_to_tensor(train_df, feature_columns)
val_tensor = utils.df_to_tensor(val_df, feature_columns)
test_tensor = utils.df_to_tensor(test_df, feature_columns)

scaler = utils.CustomSklearnScaler().fit(train_tensor)
train_tensor = scaler.transform(train_tensor)
val_tensor = scaler.transform(val_tensor)
test_tensor = scaler.transform(test_tensor)

model = utils.FitnessEenrgyModel(train_tensor.shape[-1], output_activation=nn.Identity())
model.apply(utils.init_weights)
results = utils.train_and_validate_model(
    model, train_tensor, val_tensor,
    loss_function=utils.fitness_hinge_loss,  
    loss_function_kwargs=dict(margin=4.0),
    lr=1e-2, weight_decay=0.0,
    batch_size=1, k=32,
    patience_epochs=10,
    random_seed=42,
    eval_method=utils.evaluate_fitness_flipped_sign,
)

Epoch 0: train loss 3.8357 | val loss 3.6082
Epoch 0: new best model with loss 3.6082
Epoch 0: updating patience loss from inf to 3.6082
Epoch 1: new best model with loss 3.2079
Epoch 1: updating patience loss from 3.6082 to 3.2079
Epoch 2: new best model with loss 2.7676
Epoch 2: updating patience loss from 3.2079 to 2.7676
Epoch 3: new best model with loss 2.4279
Epoch 3: updating patience loss from 2.7676 to 2.4279
Epoch 4: new best model with loss 2.1695
Epoch 4: updating patience loss from 2.4279 to 2.1695
Epoch 5: new best model with loss 2.0570
Epoch 5: updating patience loss from 2.1695 to 2.0570
Epoch 6: new best model with loss 2.0172
Epoch 6: updating patience loss from 2.0570 to 2.0172
Epoch 7: new best model with loss 2.0050
Epoch 7: updating patience loss from 2.0172 to 2.0050
Epoch 8: new best model with loss 1.8216
Epoch 8: updating patience loss from 2.0050 to 1.8216
Epoch 10: train loss 1.1250 | val loss 1.6918
Epoch 10: new best model with loss 1.6918
Epoch 10: updat

In [16]:
scoring(model, train_tensor), scoring(model, val_tensor), scoring(model, test_tensor)

({'overall_ecdf': -0.38885591947115383, 'single_game_rank': 0.541259765625},
 {'overall_ecdf': -0.437274639423077, 'single_game_rank': 0.5068359375},
 {'overall_ecdf': -0.4460673076923077,
  'single_game_rank': 0.48906248807907104})

In [20]:
train_preds = model(train_tensor).detach().squeeze().numpy()
val_preds = model(val_tensor).detach().squeeze().numpy()
test_preds = model(test_tensor).detach().squeeze().numpy()

In [28]:
((train_preds[:, 0, None] < train_preds[:, 1:]).sum(axis=1) / (train_preds.shape[1] - 1)).mean()

0.541259765625

In [ ]:
def _make_labels(prds: np.ndarray):
    labels = np.zeros_like(prds)
    labels[:, 0] = 1
    return labels

train_labels = _make_labels(train_preds)
val_labels = _make_labels(val_preds)
test_labels = _make_labels(test_preds)


In [ ]:
from sklearn import metrics


def predict_at_threshold(preds: np.ndarray, threshold: float):
    y_pred = (preds < threshold).astype(int)
    y_true = np.zeros_like(y_pred)
    y_true[:, 0] = 1
    return y_true.reshape(-1), y_pred.reshape(-1)


def evaluate_f1_with_threshold(preds: np.ndarray, threshold: float):
    return metrics.f1_score(*predict_at_threshold(preds, threshold))


def find_f1_threshold(preds: np.ndarray):
    best_f1 = 0
    best_threshold = 0
    for threshold in np.linspace(preds.min(), preds.max(), 100):
        f1 = evaluate_f1_with_threshold(preds, threshold)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1


train_threshold, train_f1 = find_f1_threshold(train_preds)
evaluate_f1_with_threshold(test_preds, train_threshold)

In [ ]:
metrics.PrecisionRecallDisplay.from_predictions(train_labels.reshape(-1), train_preds.reshape(-1))

In [ ]:
(train_preds[:, 0, np.newaxis] <= train_preds[:, 1:]).sum(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.hist(train_preds[:, 1:].reshape(-1), bins=100, label='negatives')
ax.hist(train_preds[:, 0], bins=100, label='positives')
ax.legend()
plt.show()

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
train_ecdf = ECDF(train_preds.reshape(-1))

In [ ]:
train_ecdf(train_preds[:, 0]).mean()

In [ ]:
t = torch.tensor(train_preds)

In [ ]:
p = t[:, 0]
n = t[:, 1:]

(p[:, None] < n).mean(axis=1, dtype=torch.float)